# Método de Interpolación de Newton


### Fundamento matemático

El método de interpolación de Newton consiste en dos pasos. Primero debemos calcular una serie de coeficientes que suelen ser representados en una tabla. Segundo, obtenemos el polinomio de interpolación de Newton.

El primer paso consiste en generar una tabla con n-1 columnas y n filas. Cada columna es rellenada con funciones del tipo:

$$f_k(\{x\}_{i}^{j}) = \frac{f(x_j) - f(x_i)}{x_j - x_i} \ donde \ j = i + k $$ 
 El segundo paso consiste en la obtención del Polinomio de Interpolación de Newton que tiene la forma:

$$p_i(x) = p_{i-1}(x) + f_i(\{x\}_{0}^{i}) \prod_{j=0}^{n} (x - x_j) (
x_i)$$

### Aplicación comput
aEn este caso, describiemos el método computacional empleado en un apartado posterior de nombre 'Método de Diferencias Divididas de Newton - Implementación'.cóptima.

## Funciones
* $f(x) = \sin(x)$

* $f(x) = \frac{1}{1 + 25x^2}$

* $f(x) = e^{-20x^2}$

## Bibliotecas y dependencias

In [ ]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
import sympy as spy
cmap = cm.jet

## Nodos de interpolación

In [ ]:
x_11 = np.loadtxt('nodos_11.txt', delimiter='\t')
x_21 = np.loadtxt('nodos_21.txt', delimiter='\t')
x_11_equis = np.loadtxt('nodos_equis_11.txt', delimiter='\t')
x_21_equis = np.loadtxt('nodos_equis_21.txt', delimiter='\t')

In [ ]:
#Funciones de trabajo

def y_1(x:np.array)->np.array:
  '''
  Devuelve el resultado del seno de cada elemento de x
    y = np.sin(x)
  inputs:
    x (np.array): arreglo unidimensional de valores de x en radianes

  return:
    sin(x) (np.array): seno del ángulo para cada valor de x
  '''
  return np.sin(x)

def y_2(x:np.array)->np.array:
  '''
  Devuelve el resultado de la expresión 1/(1+(25*x**2)) para cada elemento de x
    y = 1/(1+(25*x**2))
  inputs:
    x(np.array): arreglo unidimensional de valores de x

  return:
    1/(1+(25*x**2)) (np.array): resultado de la expresión dada para cada valor de x
  '''
  return 1/(1+(25*x**2))

def y_3(x:np.array)->np.array:
  '''
  Devuelve el resultado de la expresión np.exp(-20*x**2) para cada elemento de x
    y = np.exp(-20*x**2)
  inputs:
    x (np.array): arreglo unidimensional de valores de x

  return:
    np.exp(-20*x**2) (np.array): resultado de la expresión para cada valor de x
  '''

  return np.exp(-20*x**2)

### Método de Diferencias Divididas de Newton - Implementación



En la siguiente celda de ejecución podemos ver el método de diferencias de Newton implementado. Tenemos tres funciones:
- La primera función es Newton_table, encargada de obtener la tabla de Newton asociada a un conjunto de puntos x,y; que coinciden con los nodos y el efecto de la función f(x) sobre estos
- La segunda función es Newton_Poly, que a partir de la tabla obtenida con el método anterior permite calcular el polinomio de Newton asociado
- Finalmente tenemos una función encargada de representar los resultados que hemos denomindado plot_results. Además, esta función calcula el error absoluto asociado al método.

In [ ]:
error_dict_1 = {}
error_dict_2 = {}
error_dict_3 = {}
def Newton_table(x, y):
    n = len(x)
 
    input_array = np.transpose(np.array([x, y]))
    res_array= np.zeros([n, n-1])
    res_array = np.concatenate((input_array, res_array), axis=1)
    for j in range(2,n+2):
        
        for i in range(n-j+1):
    
            res_array[i,j] = (res_array[i+1,j-1]-res_array[i,j-1])/((x[i+j-1] - x[i]))
            
    return res_array
def Newton_Poly(Newton_table):
    x = spy.Symbol('x')
    poly_list =[]
    rows, n = Newton_table.shape
    p0 = Newton_table[0,1]
    
    for col in range(2, n):
        poly_str =1
        for row in range(col-1):
            
            poly_str = poly_str * (x-Newton_table[row,0])
            
        poly_str = Newton_table[0,col]* poly_str
        
        p0 += poly_str
        
    return p0     

def plot_results(x_val, y_val, poly, f, desc):
    x = spy.Symbol('x')
    
    plt.plot(x_val,y_val)

    x_to_eval =  array = np. arange(start=x_val[0], stop=x_val[-1], step=0.1)
    n = len(x_to_eval)
    
    pol_eval = np.zeros(n)
    for i in range(n):
        x_i= x_to_eval[i]
        
        pol_eval_i = poly.evalf(subs={x:x_i})
        pol_eval[i] = pol_eval_i

    err = 0
    for i in range(len(x_val)):
        x_i = x_val[i]
        
        err = err + abs(y_val[i] - poly.evalf(subs={x:x_i}))
    err_eval  =0   
    for i in range(n):
        x_i= x_to_eval[i]
        y_eval =  f(x_i)   
        err_eval = err_eval + abs(y_eval - poly.evalf(subs={x:x_i}))
    plt.plot(x_to_eval, pol_eval)
    print('El error absoluto del ajuste, en los nodos dados, es: ', err)
    print('El error absoluto del ajuste, en el intervalo de los nodos granulado en 0.1, es: ', err_eval )
    if f == y_1:
        error_dict_1.update({desc:['Error en los nodos', err, 'Error en el intervalo a paso 0.1', err_eval]})
    if f == y_2:
        error_dict_2.update({desc:['Error en los nodos', err, 'Error en el intervalo a paso 0.1', err_eval]})
    if f == y_3:
        error_dict_3.update({desc:['Error en los nodos', err, 'Error en el intervalo a paso 0.1', err_eval]})

### Función sin(x)

Cuando trabjamos con 11 nodos equiespaciados tenemos:

In [ ]:
x_val = x_11_equis
y_val = y_1(x_val)

In [ ]:
%%time
sinx_11_eq_tab = Newton_table(x_val, y_val)
sinx_11_eq_poly = Newton_Poly(sinx_11_eq_tab)



Calculamos el polinomio como:

Representado los resultados:

In [ ]:
%%time
plot_results(x_val, y_val, sinx_11_eq_poly, y_1, 'Sin(x) 11 nodos equiespaciados')

Para el caso de sin(x) con 11 nodos de Chebyshev:

In [ ]:
x_val = x_11
y_val = y_val = y_1(x_val)

In [ ]:
%%time
sinx_11_cheby_tab = Newton_table(x_val, y_val)
sinx_11_cheby_poly = Newton_Poly(sinx_11_eq_tab)


In [ ]:
%%time
plot_results(x_val, y_val, sinx_11_cheby_poly,y_1, 'Sin(x) 11 nodos de Chebyshev')

Para el caso de 21 nodos equidistantes:

In [ ]:
x_val = x_21_equis
y_val = y_1(x_val)

In [ ]:
%%time
sinx_21_eq_tab = Newton_table(x_val, y_val)
sinx_21_eq_poly = Newton_Poly(sinx_21_eq_tab)


In [ ]:
%%time
plot_results(x_val, y_val, sinx_21_eq_poly, y_1, 'Sin(x) 21 nodos equiespaciados')

Para el caso de los 21 nodos de Chebyshev:

In [ ]:
x_val = x_21
y_val = y_1(x_val)

In [ ]:
%%time
sinx_21_cheby_tab = Newton_table(x_val, y_val)
sinx_21_cheby_poly = Newton_Poly(sinx_21_cheby_tab)

In [ ]:
%%time
plot_results(x_val, y_val, sinx_21_cheby_poly, y_1, 'Sin(x) 21 nodos de Chebyshev')

### Función $f(x) = \frac{1}{1 + 25x^2}$



Para 11 nodos equidistantes

In [ ]:
x_val = x_11_equis
y_val = y_2(x_val)

In [ ]:
%%time
y25_11_eq_tab = Newton_table(x_val, y_val)

y25_11_eq_poly = Newton_Poly(y25_11_eq_tab)


In [ ]:
%%time
plot_results(x_val, y_val, y25_11_eq_poly, y_2, '1/(1+25x^2) 11 nodos equiespaciados')

Para el caso de 11 nodos de Chebyshev:

In [ ]:
x_val = x_11
y_val = y_2(x_val)

In [ ]:
%%time
y25_11_cheby_tab = Newton_table(x_val, y_val)
y25_11_cheby_poly = Newton_Poly(y25_11_cheby_tab)

In [ ]:
%%time
plot_results(x_val, y_val, y25_11_cheby_poly, y_2,'1/(1+25x^2) 11 nodos de Chebyshev')

Para el caso de 21 nodos equiespaciados:

In [ ]:
x_val = x_21_equis
y_val = y_2(x_val)

In [ ]:
%%time
y25_21_eq_tab = Newton_table(x_val, y_val)
y25_21_eq_poly = Newton_Poly(y25_21_eq_tab)

In [ ]:
%%time
plot_results(x_val, y_val, y25_21_eq_poly, y_2,'1/(1+25x^2) 21 nodos equiespaciados')

Para el caso de 21 nodos de Chebyshev:

In [ ]:
x_val = x_21
y_val = y_2(x_val)

In [ ]:
%%time
y25_21_cheby_tab = Newton_table(x_val, y_val)
y25_21_cheby_poly = Newton_Poly(y25_21_cheby_tab)

In [ ]:
%%time
plot_results(x_val, y_val, y25_21_cheby_poly, y_2, '1/(1+25x^2) 21 nodos de Chebyshev')

### Función $f(x) = e^{-20x^2}$


Para el caso de 11 nodos equidistantes:

In [ ]:
%%time
x_val = x_11_equis
y_val = y_3(x_val)

ye_11_eq_tab = Newton_table(x_val, y_val)
ye_11_eq_poly = Newton_Poly(ye_11_eq_tab)


In [ ]:
%%time
plot_results(x_val, y_val, ye_11_eq_poly, y_3, 'e^(-20*x^2) 11 nodos equiespaciados')

Para el caso de 11 nodos de Chebyshev:

In [ ]:
%%time
x_val = x_11
y_val = y_3(x_val)

ye_11_cheb_tab = Newton_table(x_val, y_val)
ye_11_cheb_poly = Newton_Poly(ye_11_cheb_tab)

In [ ]:
%%time
plot_results(x_val, y_val, ye_11_cheb_poly, y_3, 'e^(-20*x^2) 11 nodos de Chebyshev')

Para 21 nodos equidistantes se tiene:

In [ ]:
%%time
x_val = x_21_equis
y_val = y_3(x_val)

ye_21_eq_tab = Newton_table(x_val, y_val)
ye_21_eq_poly = Newton_Poly(ye_21_eq_tab)

In [ ]:
%%time
plot_results(x_val, y_val, ye_21_eq_poly, y_3, 'e^(-20*x^2) 21 nodos equiespaciados')

Para el caso de 21 nodos de Chebyshev:

In [ ]:
%%time
x_val = x_21
y_val = y_3(x_val)

ye_21_cheby_tab = Newton_table(x_val, y_val)
ye_21_cheby_poly = Newton_Poly(ye_21_cheby_tab)

In [ ]:
%%time
plot_results(x_val, y_val, ye_21_cheby_poly, y_3, 'e^(-20*x^2) 21 nodos de Chebyshev')

In [ ]:
print(error_dict_1)

In [ ]:
def plot_bars(error_dict):
    
    bar_data_nodes = []
    bar_err_nodes = []
    bar_data_eval = []
    bar_err_eval = []
    
    for i in list(error_dict.keys()):
        err_nodes, err, err_inter, err_eval =error_dict[i]
        bar_data_nodes.append(i) 
        bar_data_eval.append(i)
        bar_err_nodes.append(err)
        bar_err_eval.append(err_eval)

    
    fig = plt.figure(figsize=(10,5))
    plt.xticks(rotation=30)
    plt.bar(bar_data_nodes, bar_err_nodes, color = 'blue', width = 0.2)
    plt.xlabel('Caso a estudiar')
    plt.ylabel('Error asociado')
    plt.title('Comparación en los nodos')
    plt.show()
    fig2 = plt.figure(figsize=(10,5))
    plt.xticks(rotation=30)
    plt.bar(bar_data_eval, bar_err_eval, color = 'maroon', width = 0.2)
    plt.xlabel('Caso a estudiar')
    plt.ylabel('Error asociado')
    plt.title('Comparación en el intervalo')
    plt.show()
    print('Los errores en los nodos son: ', bar_err_nodes)
    print('Los errores en el intervalo son: ', bar_err_eval)

In [ ]:
plot_bars(error_dict_1)

Los que podemos comprobar para la función sin(x), es lo siguiente:
- Evaluando solamente el error en los nodos, para 11 nodos de Chebyshev obtenemos el máximo error. Esto es probablemente debido a que el comportamient ode la función sin(x) en el intervalo a estudiar es antisimétrica, lo que facilita el cálculo sin variación en la distancia entre nodos.
- En el caso de 21 nodos vemos que los errores son del orden del error para 11 nodos equidistantes.
- Cuando comparamos los errores en el intervalo con los erroes en los nodos, podemos concluir que la interpolación es mejor cuando trabajamos con 21 nodos frente a 11. Además, los nodos de Chebyshev permiten una interpolación 1 orden de magnitud superior a 21 nodos equiespaciados.

In [ ]:
plot_bars(error_dict_2)

Para la función 1/(1+25x^2) podemos concluir:
- Los errores evaluados en los nodos dan buenos resultados, por debajo de 10e-9. Es decir que el polinomio se ajusta muy bien a los nodos a interpolar.
- Pero además, si estudiamos el comportamiento del polinomio en todo el intervalo, comprobamos que los errores son más grandes. El mejor de todos los resultados lo proporciona Chebyshev para 21 nodos.
- El error máximo para el intervalo se encuentra en el caso de 21 nodos equiespaciados, consecuencia del comportamiento del polinomio en los extremos del intervalo.

In [ ]:
plot_bars(error_dict_3)

Para la función e^(-20*x^2) tenemos un comportamiento similar al caso anterior:
- El caso de 21 nodos de Chebyshev es el mejor para el intervalo, aunque la evaluación solamnete en los nodos ajustados es mejor para 11 ndoos equiespaciados y para 11 nodos de Chebyshev.
- De nuevo, un comportamiento erroneo del polinomio en los extremos del intervalo para 21 nodos equidistantes provoca que este sea el peor polinomio de interpolación para el intervalo.